In [1]:
!nvidia-smi

Wed Feb 22 06:38:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
!pip install -q transformers[sentencepiece] fastai fastbook ohmeow-blurr nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 KB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 KB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install -q onnxruntime onnx==1.10.0 onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 12.6 MB/s eta 0:00:00


In [6]:
import onnxruntime
print(onnxruntime.__version__)

1.14.0


In [5]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

/usr/local/lib/python3.8/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


In [ ]:
from tqdm.notebook import tqdm
import numpy

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# go to your project directory
%cd /content/drive/MyDrive/

# Data

In [9]:
df = pd.read_csv("movies_and_tvseries.csv")

In [10]:
df.shape

(25326, 3)

In [11]:
genres_list = df.genres.to_list()

genre_count = {}
for genres in genres_list:

  try:
    genre_list = eval(genres)
    for genre in genre_list:
      if genre in genre_count.keys():
        genre_count[genre] += 1
      else:
        genre_count[genre] = 1
  except:
    if genres in genre_count.keys():
      genre_count[genres] += 1
    else:
      genre_count[genres] = 1
  

print(f"Number of Genres: {len(genre_count)}")
print(genre_count)

Number of Genres: 28
{'Fantasy': 2963, 'Comedy': 5251, 'Family': 1190, 'Drama': 10244, 'Horror': 1476, 'Animation': 2735, 'Short': 264, 'Adventure': 4878, 'Sci-Fi': 1565, 'Documentary': 665, 'History': 646, 'Musical': 108, 'Music': 122, 'Thriller': 2837, 'Game-Show': 54, 'War': 358, 'Reality-TV': 175, 'Western': 195, 'Talk-Show': 31, 'Biography': 355, 'Romance': 959, 'Sport': 259, 'Mystery': 1568, 'News': 24, 'Crime': 5596, 'Action': 15030, 'Film-Noir': 6, 'Adult': 1}


In [12]:
threshhold = int(len(df) * 0.01)
rare_genres = [key for key, value in genre_count.items() if value < threshhold]
len(rare_genres), rare_genres[:5]

(9, ['Musical', 'Music', 'Game-Show', 'Reality-TV', 'Western'])

In [13]:
genres_list = df.genres.to_list()
revised_genre_list = []
indices_to_drop = []
for idx, genres in enumerate(genres_list):
  
  try:
    genre_list = eval(genres)
    revised_genres = []

    for genre in genre_list:
      if genre not in rare_genres:
        revised_genres.append(genre)
    
    if len(revised_genres) == 0:
      indices_to_drop.append(idx)

    else:
      revised_genre_list.append(revised_genres)
  except:
    revised_genres = []
    if genres not in rare_genres:
      revised_genres.append(genres)
    
    if len(revised_genres) == 0:
      indices_to_drop.append(idx)

    else:
      revised_genre_list.append(revised_genres)

df = df.drop(indices_to_drop).reset_index(drop=True)
df.shape

(25322, 3)

In [14]:
df["revised_genres"] = revised_genre_list

In [15]:
df.head(10)

,title,description,genres,revised_genres
0,A Court of Thorns and Roses,A huntress agrees to travel to a magical realm with a faerie lord in return for her family's safety.,Fantasy,[Fantasy]
1,Sabrina the Teenage Witch,"When a sixteen-year-old high school student finds out she's a witch, her two aunts offer guidance on how to control her newly-discovered magical powers.","['Comedy', 'Family', 'Fantasy']","[Comedy, Family, Fantasy]"
2,Sweet Home,"Hyun, a loner high school student who lost his entire family in a terrible accident, is forced to leave his home and has to face a new reality where monsters are trying to wipe out all of humanity. Now he must fight against all odds to try and race against the clock to save what is left of the human race before it's too late.","['Drama', 'Fantasy', 'Horror']","[Drama, Fantasy, Horror]"
3,The Amazing World of Gumball,The life of a twelve-year-old boy who happens to be a blue cat as he lives with his family and other strange creatures.,"['Animation', 'Short', 'Adventure']","[Animation, Short, Adventure]"
4,Wizards of Waverly Place,"The Russo family may be an ordinary family with an average restaurant, but behind close doors, all three children must compete to be the next family wizard.","['Adventure', 'Comedy', 'Drama']","[Adventure, Comedy, Drama]"
5,Good Witch,"""Good Witch"" will take viewers on a new magical journey with Cassie Nightingale and her daughter Grace. When Dr. Sam Radford moves in next door to Grey House with his son, they are charmed by the 'magical' mother-daughter duo.","['Comedy', 'Drama', 'Fantasy']","[Comedy, Drama, Fantasy]"
6,Motherland: Fort Salem,A trio of witches is trained to become powerful weapons for the American military.,"['Drama', 'Fantasy', 'Sci-Fi']","[Drama, Fantasy, Sci-Fi]"
7,The Shannara Chronicles,"Series of adventures, war, and evil that occur throughout the history of the Four Lands.","['Adventure', 'Drama', 'Fantasy']","[Adventure, Drama, Fantasy]"
8,Tales of the Walking Dead,"An episodic anthology that will follow individual characters from the The Walking Dead TV universe, both new and old.","['Drama', 'Fantasy', 'Horror']","[Drama, Fantasy, Horror]"
9,H2O: Just Add Water,Three girls become mermaids after a mysterious island transforms them and gives them special powers.,"['Drama', 'Family', 'Fantasy']","[Drama, Family, Fantasy]"


In [16]:
revised_genre_list = df.revised_genres.to_list()

revised_genre_count = {}
for genres in revised_genre_list:
    genre_list = genres
    for genre in genre_list:
      if genre in revised_genre_count.keys():
        revised_genre_count[genre] += 1
      else:
        revised_genre_count[genre] = 1
  

print(f"Number of Genres: {len(revised_genre_count)}")
print(revised_genre_count)

Number of Genres: 19
{'Fantasy': 2963, 'Comedy': 5251, 'Family': 1190, 'Drama': 10244, 'Horror': 1476, 'Animation': 2735, 'Short': 264, 'Adventure': 4878, 'Sci-Fi': 1565, 'Documentary': 665, 'History': 646, 'Thriller': 2837, 'War': 358, 'Biography': 355, 'Romance': 959, 'Sport': 259, 'Mystery': 1568, 'Crime': 5596, 'Action': 15030}


In [17]:
encode_genre_types = { key: idx  for idx, (key, value) in enumerate(revised_genre_count.items()) }
with open("genre_types_encoded.json", "w") as fp:
  json.dump(encode_genre_types, fp)

In [18]:
categorical_genre_list = []
revised_genres_list = df.revised_genres.to_list()

for revised_genres in revised_genres_list:
  categorical_list = [0] * (len(encode_genre_types))
  for genre in revised_genres:
    genre_type_index = encode_genre_types[genre]
    categorical_list[genre_type_index] = 1
  categorical_genre_list.append(categorical_list)

categorical_genre_list[3][:19]

[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [19]:
df["genre_cat_list"] = categorical_genre_list
df.head()

,title,description,genres,revised_genres,genre_cat_list
0,A Court of Thorns and Roses,A huntress agrees to travel to a magical realm with a faerie lord in return for her family's safety.,Fantasy,[Fantasy],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,Sabrina the Teenage Witch,"When a sixteen-year-old high school student finds out she's a witch, her two aunts offer guidance on how to control her newly-discovered magical powers.","['Comedy', 'Family', 'Fantasy']","[Comedy, Family, Fantasy]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Sweet Home,"Hyun, a loner high school student who lost his entire family in a terrible accident, is forced to leave his home and has to face a new reality where monsters are trying to wipe out all of humanity. Now he must fight against all odds to try and race against the clock to save what is left of the human race before it's too late.","['Drama', 'Fantasy', 'Horror']","[Drama, Fantasy, Horror]","[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,The Amazing World of Gumball,The life of a twelve-year-old boy who happens to be a blue cat as he lives with his family and other strange creatures.,"['Animation', 'Short', 'Adventure']","[Animation, Short, Adventure]","[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Wizards of Waverly Place,"The Russo family may be an ordinary family with an average restaurant, but behind close doors, all three children must compete to be the next family wizard.","['Adventure', 'Comedy', 'Drama']","[Adventure, Comedy, Drama]","[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [20]:
encode_genre_types

{'Fantasy': 0,
 'Comedy': 1,
 'Family': 2,
 'Drama': 3,
 'Horror': 4,
 'Animation': 5,
 'Short': 6,
 'Adventure': 7,
 'Sci-Fi': 8,
 'Documentary': 9,
 'History': 10,
 'Thriller': 11,
 'War': 12,
 'Biography': 13,
 'Romance': 14,
 'Sport': 15,
 'Mystery': 16,
 'Crime': 17,
 'Action': 18}

# Data Split

In [21]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(22790, 2532)

In [22]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title,description,genres,revised_genres,genre_cat_list
6848,Super Veer,"Meet once again Ishani, Ishanvi Chakravyu last part not complet story so the Super Veer resolve the story of Chakravyu and Ishani, Ishanvi Come out the game. On Park and.","['Adventure', 'Drama', 'Fantasy']","[Adventure, Drama, Fantasy]","[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5758,Ryan Defrates: Secret Agent,"""Ryan Defrates: Secret Agent"" is an animated action-adventure series about a young spy with a whole lot to learn! He's arrogant and reckless and always insists on working alone. That is, ... See full summary »","['Animation', 'Action', 'Adventure']","[Animation, Action, Adventure]","[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
6438,The Borros Saga: Banesbreak,An unsuspecting crew of adventurers as they find themselves ripped away from the familiar and into the troubled past.,"['Adventure', 'Fantasy']","[Adventure, Fantasy]","[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
20413,Sky on Fire,"A security guard becomes embroiled in a battle to protect stem cell research, which can potentially cure cancer, from falling into the wrong hands.","['Action', 'Crime', 'Thriller']","[Action, Crime, Thriller]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1]"
2062,Chimera,"An explosion took place in the past. That explosion led to a serial murder case known as the Chimera case. 35 years later, a similar explosion takes place.","['Mystery', 'Thriller']","[Mystery, Thriller]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0]"


# Fastai and Blurr Inference

In [27]:
model_path = "models/plot-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [32]:
learner_inf.blurr_predict("Just checking random plot.")

[{'labels': [],
  'scores': [],
  'class_indices': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'class_labels': ['Fantasy', 'Comedy', 'Family', 'Drama', 'Horror', 'Animation', 'Short', 'Adventure', 'Sci-Fi', 'Documentary', 'History', 'Thriller', 'War', 'Biography', 'Romance', 'Sport', 'Mystery', 'Crime', 'Action'],
  'probs': [0.19995570182800293,
   0.38203293085098267,
   0.020907917991280556,
   0.1748020499944687,
   0.08381219953298569,
   0.0231977179646492,
   0.023485973477363586,
   0.043086081743240356,
   0.04761049523949623,
   0.003294676309451461,
   0.008804365061223507,
   0.10242342203855515,
   0.0011638951255008578,
   0.0008886588038876653,
   0.020745109766721725,
   0.000279721716651693,
   0.44646117091178894,
   0.23549775779247284,
   0.1857810765504837]}]

# Evaluation

In [37]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df["genre_cat_list"].to_list()]
  outputs = [np.asarray(pred) for pred in preds]

  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average="micro")
  f1_score_macro = metrics.f1_score(targets, outputs, average="macro")

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [38]:
preds = []

for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row["description"]
  labels = learner_inf.blurr_predict(desc)[0]["labels"]
  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

preds[0][:20]

  0%|          | 0/2532 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [40]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.6209483504857171
F1 Score (Macro) = 0.42884109501985906


# Convert to ONNX

In [41]:
model_path = "models/plot-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [ ]:
learner_inf.model

In [44]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    "models/plot-classifier.onnx",
    input_names=["input_ids"],
    output_names=["output"],
    opset_version=13,
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_len"},
        "output": {0: "batch_size"}
    }
)

In [45]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = "models/plot-classifier.onnx"
quantized_onnx_model_path = "models/plot-classifier-quantized.onnx"

quantize_dynamic(
  onnx_model_path,
  quantized_onnx_model_path,
  weight_type=QuantType.QUInt8
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

# ONNX Inference

## Normal ONNX

In [46]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession("models/plot-classifier.onnx")
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [49]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row["description"]
  input_ids = tokenizer(desc)["input_ids"][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/2532 [00:00<?, ?it/s]

In [50]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.6206962877476438
F1 Score (Macro) = 0.4329185936407845


## Quantized ONNX

In [51]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession("models/plot-classifier-quantized.onnx")
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [52]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row["description"]
  input_ids = tokenizer(desc)["input_ids"][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/2532 [00:00<?, ?it/s]

In [53]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.6193039800038453
F1 Score (Macro) = 0.42675406564282564
